# Objective:

1. Explore the best linear method
2. Perserve Interpretability

# Code

In [181]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
warnings.filterwarnings("ignore")

In [182]:
df=pd.read_csv('train_weekly.csv')
df['Time']=pd.to_datetime(df['Time'])#,format='%Y-%m-%d')
df=df.iloc[:,:]
df=df.drop(['Signal'],axis=1)
dp=np.zeros(len(df))
pp=np.zeros(len(df))
for i in range(1,len(df)):
    dp[i]=df['AM_NetPos'][i]-df['AM_NetPos'][i-1]
    pp[i]=df['AM_NetPos'][i-1]
df['DP']=dp
df['Prev_pos']=pp
df=df.drop(["Lev_NetPos","AM_n_L","AM_n_S","Lev_n_L","Lev_n_S","AM_L","AM_S","Lev_L","Lev_S",'AM_NetPos','Time'],axis=1).drop(0)
df

,Unnamed: 0,FedFunds,1Y OIS,Libor 3 mo,Libor 6 mo,Libor 1Y,T10Y3M,T10Y2Y,T10YIE,JNK_Price,...,FF_Mkt,FF_SMB,FF_HML,FF_MOM,SPY Flows,IVV,JNK_Volume,Flag,DP,Prev_pos
1,1,0.0018,0.003070,0.00538,0.00747,0.011750,0.0305,0.0247,0.0198,0.012095,...,-0.0163,-0.0034,-0.0073,-0.0060,-3168.3790,195.4229,704400,3,12588.0,263511.0
2,2,0.0015,0.002915,0.00533,0.00751,0.011744,0.0282,0.0236,0.0183,-0.012211,...,-0.0323,-0.0065,-0.0127,-0.0194,-4424.6870,50.6436,522060,4,-17663.0,276099.0
3,3,0.0018,0.002850,0.00531,0.00742,0.011631,0.0278,0.0233,0.0169,0.012721,...,0.0033,-0.0198,0.0011,-0.0043,-10835.6820,-439.0648,821100,5,-1699.0,258436.0
4,4,0.0017,0.002760,0.00526,0.00733,0.011450,0.0300,0.0248,0.0187,0.019381,...,0.0176,0.0160,0.0063,0.0050,4171.1670,315.4354,901860,6,23534.0,256737.0
5,5,0.0018,0.002490,0.00513,0.00711,0.011019,0.0282,0.0237,0.0170,0.001285,...,0.0123,0.0065,0.0006,0.0055,-1055.8070,5.1258,787320,7,-4359.0,280271.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,419,0.0191,0.023105,0.02335,0.02523,0.028080,0.0093,0.0032,0.0211,0.002522,...,0.0022,-0.0135,0.0040,-0.0036,1915.2560,-477.6230,2431980,421,-3644.0,94621.0
420,420,0.0191,0.023340,0.02349,0.02531,0.028278,0.0093,0.0029,0.0212,0.004472,...,0.0051,0.0083,-0.0110,0.0004,4568.7080,-662.1940,3420120,422,-6691.0,90977.0
421,421,0.0191,0.023490,0.02341,0.02522,0.028243,0.0092,0.0030,0.0213,0.004220,...,0.0029,0.0000,-0.0017,0.0033,-6345.8700,401.6125,3159420,423,6860.0,84286.0
422,422,0.0191,0.023430,0.02315,0.02512,0.028135,0.0081,0.0026,0.0209,-0.002226,...,0.0069,0.0029,0.0025,0.0040,1128.0550,1505.0095,3333120,424,-5541.0,91146.0


In [183]:
y=df['DP']
X=df.drop(['DP'],axis=1)


In [184]:
import statsmodels.api as sm
X = sm.add_constant(X)
model_OLS = sm.OLS(y, X).fit()
print(model_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                     DP   R-squared:                       0.548
Model:                            OLS   Adj. R-squared:                  0.515
Method:                 Least Squares   F-statistic:                     16.43
Date:                Mon, 07 Dec 2020   Prob (F-statistic):           3.48e-51
Time:                        03:22:55   Log-Likelihood:                -4409.8
No. Observations:                 423   AIC:                             8880.
Df Residuals:                     393   BIC:                             9001.
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          8018.4784   5726.036      1.400

**Variable selection in another notebook**

In [185]:
selected=['Prev_pos', 'GOLD_price', 'JNK_Volume', 'NASDAQCOM','WILL5000INDFC', 'SPX', 'Libor 6 mo']
selected

['Prev_pos',
 'GOLD_price',
 'JNK_Volume',
 'NASDAQCOM',
 'WILL5000INDFC',
 'SPX',
 'Libor 6 mo']

# RF

In [195]:
from sklearn.ensemble import RandomForestRegressor
MSE=[]
APE=[]
start = time.time()
for i in range(350,len(df)):
    X_train=df.drop(['DP'],axis=1).iloc[:i,:]
    y_train=df['DP'][:i]
    X_val=df.drop(['DP'],axis=1).iloc[:i+1,:]
    y_val=df['DP'].iloc[i]
    model_RF=RandomForestRegressor(n_estimators=20,random_state=0,criterion='mae',
                                   min_samples_leaf = 2).fit(X_train,y_train)

    mse=(y_val-model_RF.predict(X_val)[-1])**2
    ape=np.abs((y_val-model_RF.predict(X_val)[-1])/(df['DP'].iloc[i]+df['Prev_pos'].iloc[i]))
    MSE.append(mse)
    APE.append(ape)
end = time.time()
print('RF: ')
print('MSE is ',np.mean(MSE))
print('Average Percentage Error is ',np.mean(APE))
print('Time elapsed: ',end-start)

RF: 
MSE is  103237258.47678083
Average Percentage Error is  0.05828265828079718
Time elapsed:  30.07499885559082


# VAR

In [187]:
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.tsa.api import VAR

In [194]:

MSE=[]
APE=[]
start = time.time()
for i in range(350,len(df)):
    #assume stationarity at first
    y_val=df['DP'].iloc[i]
    mod = VAR(df.iloc[:i,:]).fit(4)
    prediction=mod.forecast(np.array(df.iloc[:i,:]),1)
    pos_index=[i for i, j in enumerate(df.columns) if j == 'DP']
    y_predict=prediction[0][pos_index]
    mse=(y_val-y_predict)**2
    ape=np.abs((y_val-y_predict)/(df['DP'].iloc[i]+df['Prev_pos'].iloc[i]))
    MSE.append(mse)
    APE.append(ape)
end = time.time()
print('VAR: ')
print('MSE is ',np.mean(MSE))
print('Average Percentage Error is ',np.mean(APE))
print('Time elapsed: ',end - start)

VAR: 
MSE is  207109486.2586584
Average Percentage Error is  0.0924592835676424
Time elapsed:  2.5173985958099365


# After Variable Selection

In [189]:
# deal with the extra lag for positioning
Varmax_df=df[np.append(selected,'DP')]
#dpos = (pos-pos.shift(1)).dropna()
#Varmax_df['dpos']=dpos


In [205]:
MSE=[]
APE=[]
start = time.time()
for i in range(350,len(Varmax_df)):
    #assume stationarity at first
    y_val=Varmax_df['DP'].iloc[i]
    mod = VAR(Varmax_df.iloc[:i,:]).fit(3)
    prediction=mod.forecast(np.array(Varmax_df.iloc[:i,:]),1)
    pos_index=[i for i, j in enumerate(Varmax_df.columns) if j == 'DP']
    y_predict=prediction[0][pos_index]
    mse=(y_val-y_predict)**2
    ape=np.abs((y_val-y_predict)/(df['DP'].iloc[i]+df['Prev_pos'].iloc[i]))
    MSE.append(mse)
    APE.append(ape)
    
end = time.time()
print('VAR Selection: ')
print('MSE is ',np.mean(MSE))
print('Average Percentage Error is ' , np.mean(APE) )
print('Time elapsed: ',end-start)

VAR Selection: 
MSE is  144321860.75950733
Average Percentage Error is  0.07275392492502177
Time elapsed:  0.2747964859008789


# Transformation

In [197]:
from sklearn.preprocessing import Normalizer

In [286]:
MSE=[]
APE=[]
start = time.time()
for i in range(350,len(Varmax_df)):
    #assume stationarity at first
    y_val=Varmax_df['DP'].iloc[i]
    transformer = Normalizer().fit(Varmax_df.iloc[:i,:].drop(['DP'],axis=1))
    Trans=transformer.transform(Varmax_df.iloc[:i,:].drop(['DP'],axis=1))
    trans_df=np.append(Trans,np.array(Varmax_df['DP'].iloc[:i]).reshape(i,1),1)
    mod = VAR(trans_df).fit(3)
    prediction=mod.forecast(trans_df,1)
    pos_index=[i for i, j in enumerate(trans_df) if j == 'DP']
    y_predict=prediction[0][-1]
    mse=(y_val-y_predict)**2
    ape=np.abs((y_val-y_predict)/(df['DP'].iloc[i]+df['Prev_pos'].iloc[i]))
    MSE.append(mse)
    APE.append(ape)
    
end = time.time()
print('VAR Transformation: ')
print('MSE is ',np.mean(MSE))
print('Average Percentage Error is ' , np.mean(APE) )
print('Time elapsed: ',end-start)

VAR Transformation: 
MSE is  147515330.53766423
Average Percentage Error is  0.0755981607013019
Time elapsed:  1.133061170578003


# VMA

In [279]:
def nearestPD(A):
    """Find the nearest positive-definite matrix to input

    A Python/Numpy port of John D'Errico's `nearestSPD` MATLAB code [1], which
    credits [2].

    [1] https://www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd

    [2] N.J. Higham, "Computing a nearest symmetric positive semidefinite
    matrix" (1988): https://doi.org/10.1016/0024-3795(88)90223-6
    """

    B = (A + A.T) / 2
    _, s, V = la.svd(B)

    H = np.dot(V.T, np.dot(np.diag(s), V))

    A2 = (B + H) / 2

    A3 = (A2 + A2.T) / 2

    if isPD(A3):
        return A3

    spacing = np.spacing(la.norm(A))
    I = np.eye(A.shape[0])
    k = 1
    while not isPD(A3):
        mineig = np.min(np.real(la.eigvals(A3)))
        A3 += I * (-mineig * k**2 + spacing)
        k += 1

    return A3

def isPD(B):
    """Returns true when input is positive-definite, via Cholesky"""
    try:
        _ = la.cholesky(B)
        return True
    except la.LinAlgError:
        return False

In [303]:
MSE=[]
APE=[]
start = time.time()
for i in range(350,len(Varmax_df)):
    y_val=Varmax_df['DP'].iloc[i]
    mod = sm.tsa.VARMAX(Varmax_df.iloc[:i,[0,1,7]], order=(0,2), error_cov_type='diagonal')
    res = mod.fit(maxiter=1000, disp=False)
    prediction=res.forecast(1)
    #pos_index=[i for i, j in enumerate(Varmax_df.columns) if j == 'DP']
    y_predict=prediction['DP'].iloc[0]
    mse=(y_val-y_predict)**2
    ape=np.abs((y_val-y_predict)/(df['DP'].iloc[i]+df['Prev_pos'].iloc[i]))
    MSE.append(mse)
    APE.append(ape)
    
end = time.time()
print('VARMAX Selection: ')
print('MSE is ',np.mean(MSE))
print('Average Percentage Error is ' , np.mean(APE) )
print('Time elapsed: ',end-start)

VARMAX Selection: 
MSE is  896238274843.8809
Average Percentage Error is  0.806224205523157
Time elapsed:  101.65589642524719


# Transformation

In [304]:
MSE=[]
APE=[]
start = time.time()
transformer = Normalizer().fit(Varmax_df.iloc[:,:].drop(['DP'],axis=1))
Trans=transformer.transform(Varmax_df.iloc[:,:].drop(['DP'],axis=1))
trans_df=np.append(Trans,np.array(Varmax_df['DP']).reshape(len(Trans),1),1)
                   
for i in range(350,len(Varmax_df)):
    y_val=Varmax_df['DP'].iloc[i]
    mod = sm.tsa.VARMAX(trans_df[:i,[0,2,7]],order=(1,2), error_cov_type='diagonal')
    res = mod.fit()
    #pos_index=[i for i, j in enumerate(trans_df) if j == 'DP']
    y_predict=prediction['DP'].iloc[0]
    mse=(y_val-y_predict)**2
    ape=np.abs((y_val-y_predict)/(df['DP'].iloc[i]+df['Prev_pos'].iloc[i]))
    MSE.append(mse)
    APE.append(ape)
    
end = time.time()
print('VAR Transformation: ')
print('MSE is ',np.mean(MSE))
print('Average Percentage Error is ' , np.mean(APE) )
print('Time elapsed: ',end-start)

VAR Transformation: 
MSE is  143936256.62997085
Average Percentage Error is  0.07237884979239888
Time elapsed:  61.41954278945923
